<a href="https://colab.research.google.com/github/oviyajay/ML-Final-Project/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import np from numpy
import tensorflow as tf
import os 
import cv2

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/



In [ ]:
#Eric 

In [ ]:
#Iant

In [ ]:
#Anindita
def label_img(img): 
    word_label = img.split('.')[-3] 
    # DIY One hot encoder 
    if word_label == 'mask': return [1, 0] 
    elif word_label == 'no mask': return [0, 1] 

In [ ]:
#Oviya